In [29]:
import akshare as ak
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)

gc001 = ak.bond_zh_hs_daily(symbol="sh204001")
gc001['date'] = gc001['date'].apply(pd.to_datetime)

In [208]:
years = ['2017','2018','2019','2020','2021','2022','2023']
modes = ['high','low','open','close']

result = pd.DataFrame(columns = ['year'] + modes)
result['year'] = years
result = result.set_index('year')

for year in years:
    for mode in modes:
        initials = 1000
        interest = 0
        start_date = year + '-01-01'
        end_date = year +'-12-31'

        for row, i in gc001[gc001['date'].isin(pd.date_range(start=start_date, end=end_date))].iterrows():
            investable = initials + (interest//1000)*1000
            day_rate = i[mode]
            week_of_day = i['date'].isoweekday()
            if week_of_day == 4:
                day_interest = investable * day_rate * 3/ (360*100) - 3 * 0.001 * investable /100
            else:
                day_interest = investable * day_rate * 1/ (360*100) - 1 * 0.001 * investable /100
            day_interest = day_interest if day_interest > 0 else 0 
            interest += day_interest

        day_to_now = (datetime.today() - datetime(int(year),1,1)).days 
        day_to_now = 365 if day_to_now > 365 else day_to_now
        bentchmark = day_to_now/365
        year_rate = interest/initials*100/bentchmark

        result[mode][year] = "{0:.2f}%".format(year_rate)

In [209]:
result[['high','open']]


,high,open
year,,
2017,5.78%,4.02%
2018,3.50%,2.79%
2019,2.51%,2.21%
2020,2.22%,1.92%
2021,2.30%,2.00%
2022,1.81%,1.66%
2023,2.36%,2.05%
